In [127]:
import torch
from torch.nn import Module, Embedding, Linear
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
import graphnn
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd

from tqdm import tqdm_notebook as tqdm

from time import time
import json
from glob import glob
from pprint import pprint
import io
import os
from tqdm import tqdm_notebook as tqdm

import unicodedata as ud

from collections import Counter

latin_letters= {}

def is_latin(uchr):
    try: return latin_letters[uchr]
    except KeyError:
         return latin_letters.setdefault(uchr, 'LATIN' in ud.name(uchr))

def only_roman_chars(unistr):
    return all(is_latin(uchr)
           for uchr in unistr
           if uchr.isalpha())

In [128]:
only_roman_chars("")

True

In [129]:
os.path.basename(json_files[0])[:-5]

'18350465'

In [130]:
json_files = glob('data/cvl/*.json')

def load_file(fname):
    return json.load(io.open(fname, 'r', encoding='utf-8-sig'))

data = {os.path.basename(fname)[:-5]: load_file(fname) for fname in tqdm(json_files)}

len(data)

2327

In [131]:
data = {key: value for key, value in data.items() if only_roman_chars(value.get('ARTICLE_DOMAIN', '') + value.get('ARTICLE_TITLE', ''))} 

In [132]:
len(data)

2273

In [163]:
def split_domain(domain):
    domain = domain.split(' | ')
    if len(domain) >= 3:
        return domain[-2].replace(':', ''), domain[-1].replace(':', '')
    if len(domain) == 2:
        return domain[-1].replace(':', ''), domain[-1].replace(':', '')
    if len(domain) == 1:
        return domain[0].replace(':', ''), domain[0].replace(':', '')


In [175]:
link_text_pre = "www.capitaliq.com/CIQDotNet/CreditResearch/SPResearch.aspx?DocumentId="
link_text_post = "&From=SNP_CRS"
inner_link_pre = "www.capitaliq.com"
# "LINK": "www.capitaliq.com/CIQDotNet/CreditResearch/SPResearch.aspx?DocumentId=40944002&From=SNP_CRS"

c = Counter()
d = Counter()
cd = Counter()

def convert(key, value):
    
    content = ' '.join(block['ANNOTATED_TEXT'] for block in value['EXTRACTED_CONTENT'])
    refs = [{'LINK':f"{inner_link_pre}{block['LINK']}", 
              'LINK_TEXT':block['LINK_TEXT'], 
              'ART_OBJECTID':block['ART_OBJECTID']} for block in value['ALL_ARTICLE_LINKS']]
    title = value.get('ARTICLE_TITLE', 'ERROR')
    domain = value.get('ARTICLE_DOMAIN', 'ERROR')
    domain1, domain2 = 'ERROR', 'ERROR'
    if domain == 'ERROR':
        if 'Table Of Contents' in title:
            domain1 = 'Table Of Contents'
            domain2 = 'Table Of Contents'
    else:
        domain2, domain1 = split_domain(domain)
        
    c[domain2] += 1
    d[domain1] += 1
    cd[(domain2, domain1)] += 1
    
    pub_date = value['ARTICLE_PUB_DATE']
    link = f"{link_text_pre}{key}{link_text_post}"
    article_id = key
    return {'aid': article_id, 'title':title, 'top_domain':domain2, 'bottom_domain':domain1,  'pub_date':pub_date, 'link':link, 'content':content, 'refs':refs}


new_data = {item[0] : convert(*item) for item in tqdm(data.items())}

In [183]:
print(c)
print(len(c))

Counter({'Structured Finance': 993, 'Corporates': 362, 'Insurance': 250, 'Financial Institutions': 246, 'Governments': 231, 'General Criteria': 110, 'Legal Criteria': 65, 'Table Of Contents': 13, 'Guidance': 1, 'ERROR': 1, 'Credit FAQ': 1})
11


In [184]:
print(d)
print(len(d))

Counter({'General': 346, 'RMBS': 321, 'Request for Comment': 237, 'ABS': 187, 'CDOs': 184, 'Industrials': 144, 'U.S. Public Finance': 128, 'General Criteria': 91, 'Banks': 71, 'CMBS': 69, 'Legal Criteria': 65, 'Fixed-Income Funds': 49, 'Life': 44, 'Covered Bonds': 41, 'Specialty': 39, 'Project Finance': 35, 'Property/Casualty': 34, 'International Public Finance': 31, 'Sovereigns': 28, 'Utilities': 21, 'Health': 16, 'ABCP': 15, 'Table Of Contents': 13, 'Bond': 12, 'Other': 12, 'Finance Companies': 11, 'Recovery': 11, 'Servicer Evaluations': 7, 'Broker-Dealers': 6, 'Fundamentals': 2, 'Legal': 1, 'ERROR': 1, 'Credit FAQ': 1})
33


In [185]:
print(cd)
print(len(cd))

Counter({('Structured Finance', 'RMBS'): 321, ('Structured Finance', 'ABS'): 187, ('Structured Finance', 'CDOs'): 184, ('Corporates', 'Industrials'): 144, ('Governments', 'U.S. Public Finance'): 128, ('Corporates', 'General'): 107, ('Structured Finance', 'Request for Comment'): 99, ('General Criteria', 'General Criteria'): 90, ('Insurance', 'General'): 89, ('Financial Institutions', 'General'): 73, ('Financial Institutions', 'Banks'): 71, ('Structured Finance', 'CMBS'): 69, ('Structured Finance', 'General'): 69, ('Legal Criteria', 'Legal Criteria'): 65, ('Financial Institutions', 'Fixed-Income Funds'): 49, ('Corporates', 'Request for Comment'): 44, ('Insurance', 'Life'): 44, ('Structured Finance', 'Covered Bonds'): 41, ('Insurance', 'Specialty'): 39, ('Governments', 'Request for Comment'): 36, ('Corporates', 'Project Finance'): 35, ('Insurance', 'Property/Casualty'): 34, ('Governments', 'International Public Finance'): 31, ('Governments', 'Sovereigns'): 28, ('Financial Institutions', '

In [186]:
# new_data['18178692']

In [187]:
for nd in new_data:
    if new_data[nd]['top_domain'] == 'ERROR':
        print('TDOMAIN', nd)
    if new_data[nd]['bottom_domain'] == 'ERROR':
        print('BDOMAIN', nd)
    if new_data[nd]['title'] == 'ERROR':
        print('TITLE ', nd)
#     print()

TITLE  18078955
TDOMAIN 40509634
BDOMAIN 40509634


In [188]:
for key, value in tqdm(new_data.items()):
    with open(f"data/cvl_formatted/{key}.json", 'w') as json_file:
        json.dump(value, json_file)